# PyNutrien Demo/Development notebook

## Setup

1) Install AWS CLI and set secrets in `~/.aws/credentials`
2) Set profile region in `~/.aws/config` -- TODO see link



In [ ]:

# !pip install -r requirements.txt


## Setup Validation

In [12]:
import os
import sys
from pathlib import Path
#lib_path = str(Path(os.getcwd()).parent.absolute().resolve())
#sys.path.insert(0, lib_path)

['/home/glue_user/workspace', '/home/glue_user/workspace/jupyter_workspace', '/home/glue_user/aws-glue-libs/PyGlue.zip', '/home/glue_user/spark/python/lib/py4j-0.10.9-src.zip', '/home/glue_user/spark/python', '/usr/lib64/python37.zip', '/usr/lib64/python3.7', '/usr/lib64/python3.7/lib-dynload', '', '/home/glue_user/.local/lib/python3.7/site-packages', '/usr/lib64/python3.7/site-packages', '/usr/lib/python3.7/site-packages', '/home/glue_user/.local/lib/python3.7/site-packages/IPython/extensions', '/home/glue_user/.ipython']


In [15]:
import pynutrien

from pynutrien.aws.glue import GlueSparkContext

context = GlueSparkContext()
context.spark_session # show SparkUI

## Imports and Dependencies

In [ ]:
import pynutrien

## Library Usage Examples

### Catalog Reader Example

In [16]:

from pynutrien.aws.glue import GlueSparkContext
from pynutrien.aws.glue.catalog import GlueReader

context = GlueSparkContext()
Reader = GlueReader(context.glue_context, None)
df = (
    Reader.catalog("crawler_test", "caseware_customer_a")
        .transformation_ctx("cases")
        .dataframe()
)
df.show()

+-----------+-----------+------+-------------+----------+---------------+--------------+-----+-----------------+------------+-------+----------------+----+
|primary_key|       name|gender|customer_type|birth_date|    personal_id|   card_number|email|     phone_number|country_code|address|meta_last_update|year|
+-----------+-----------+------+-------------+----------+---------------+--------------+-----+-----------------+------------+-------+----------------+----+
|     135567|  T. Oakman|      |       Active|          | SIN: 154720759|Not Applicable|     |   1 822 435 3970|          CA|       |      2008-08-12|2017|
|     135635|B. Kanahele|      |     Prospect|1959-08-20| SIN: 755909975|Not Applicable|     |            -2865|          CA|       |      2008-08-08|2017|
|     135844|  B. Wandel|     M|       Active|1956-06-15|NINO: VT931050C|Not Applicable|     |                 |          CA|       |      2008-01-05|2017|
|     136073|    S. Naff|     M|       Active|1981-06-22| SSN: 8

### Job

In [20]:
# Add arguments
import sys
sys.argv.extend(["--cfg_file_path", "dummy", "--env_file_path", "dummy", "--abc", "123"])

The below part of the code can be only run from docker container. Run the 'util/docker_run_notebook.sh' first.

In [21]:

from pynutrien.aws.glue import GlueJob

class MyGlueJob(GlueJob):
    job_name = "TEST_GLUE"
    arguments = ["abc"]
    input_path="s3://awsglue-datasets/examples/us-legislators/all/persons.json"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def extract(self,):
        
        self.dynamicframe = self.glue_context.create_dynamic_frame.from_options(
            connection_type="s3",
            connection_options={"paths": [self.input_path], "recurse": True},
            format="json",
        )

    def transform(self):
        pass

    def load(self):
        pass
    
    

job = MyGlueJob()
job.run()


2022-11-30 14:58:27.318 [INFO] [credentials:1223] MainThread:botocore.credentials - Found credentials in shared credentials file: ~/.aws/credentials
2022-11-30 14:58:27.610 [INFO] [logger:84] MainThread:TEST_GLUE - Starting: TEST_GLUE
2022-11-30 14:58:27.611 [INFO] [logger:84] MainThread:TEST_GLUE - Starting: setup
2022-11-30 14:58:27.614 [INFO] [job:79] MainThread:TEST_GLUE - Supplied Arguments: ['/home/glue_user/.local/lib/python3.7/site-packages/ipykernel_launcher.py', '-f', '/home/glue_user/.local/share/jupyter/runtime/kernel-355a1265-2c57-4bab-9cf3-915a95661778.json', '--cfg_file_path', '', '--env_file_path', '', '--abc', '123', '--cfg_file_path', 'dummy', '--env_file_path', 'dummy', '--abc', '123']
2022-11-30 14:58:27.615 [INFO] [job:80] MainThread:TEST_GLUE - Parsed Arguments: {'job_bookmark_option': 'job-bookmark-disable', 'job_bookmark_from': None, 'job_bookmark_to': None, 'JOB_ID': None, 'JOB_RUN_ID': None, 'SECURITY_CONFIGURATION': None, 'encryption_type': None, 'enable_data

### DynamoDB

In [31]:
import boto3
from pynutrien.aws.dynamodb import DynamoDB,DynamoDBPandas

session = boto3.session.Session()
client = session.client("dynamodb")
# response = client.create_table(
#     AttributeDefinitions=[
#         {
#             "AttributeName": "Artist",
#             "AttributeType": "S",
#         },
#         {
#             "AttributeName": "SongTitle",
#             "AttributeType": "S",
#         },
#     ],
#     KeySchema=[
#         {
#             "AttributeName": "Artist",
#             "KeyType": "HASH",
#         },
#         {
#             "AttributeName": "SongTitle",
#             "KeyType": "RANGE",
#         },
#     ],
#     ProvisionedThroughput={
#         "ReadCapacityUnits": 5,
#         "WriteCapacityUnits": 5,
#     },
#     TableName="Music",
# )
waiter = client.get_waiter("table_exists")
waiter.wait(TableName="Music", WaiterConfig={"Delay": 1, "MaxAttempts": 100})
response = client.batch_write_item(
    RequestItems={
        "Music": [
            {
                "PutRequest": {
                    "Item": {
                        "AlbumTitle": {
                            "S": "Somewhat Famous",
                        },
                        "Artist": {
                            "S": "No One You Know",
                        },
                        "SongTitle": {
                            "S": "Call Me Today",
                        },
                    },
                },
            },
            {
                "PutRequest": {
                    "Item": {
                        "AlbumTitle": {
                            "S": "Songs About Life",
                        },
                        "Artist": {
                            "S": "Acme Band",
                        },
                        "SongTitle": {
                            "S": "Happy Day",
                        },
                    },
                },
            },
            {
                "PutRequest": {
                    "Item": {
                        "AlbumTitle": {
                            "S": "Blue Sky Blues",
                        },
                        "Artist": {
                            "S": "No One You Know",
                        },
                        "SongTitle": {
                            "S": "Scared of My Shadow",
                        },
                    },
                },
            },
        ],
    },
)
response = client.put_item(
    Item={
        "AlbumTitle": {
            "S": "Somewhat Famous",
        },
        "Artist": {
            "S": "No One You Know",
        },
        "SongTitle": {
            "S": "Call Me Today",
        },
    },
    ReturnConsumedCapacity="TOTAL",
    TableName="Music",
)
my_dyno = DynamoDB(session)
response_get = client.get_item(
    Key={
        "Artist": {
            "S": "Acme Band",
        },
        "SongTitle": {
            "S": "Happy Day",
        },
    },
    TableName="Music",
)
my_dyno_get = my_dyno.get_item(
    key={
        "Artist": {
            "S": "Acme Band",
        },
        "SongTitle": {
            "S": "Happy Day",
        },
    },
    table_name="Music",
)
assert my_dyno_get == response_get["Item"]
update_response = my_dyno.update_item(
    ExpressionAttributeNames={
        "#AT": "AlbumTitle",
        "#Y": "Year",
    },
    ExpressionAttributeValues={
        ":t": {
            "S": "Louder Than Ever",
        },
        ":y": {
            "N": "2015",
        },
    },
    key={
        "Artist": {
            "S": "Acme Band",
        },
        "SongTitle": {
            "S": "Happy Day",
        },
    },
    ReturnValues="ALL_NEW",
    table_name="Music",
    UpdateExpression="SET #Y = :y, #AT = :t",
)
assert update_response["Attributes"] == {
    "AlbumTitle": {"S": "Louder Than Ever"},
    "Artist": {"S": "Acme Band"},
    "Year": {"N": "2015"},
    "SongTitle": {"S": "Happy Day"},
}
scan_response = my_dyno.scan_table(table_name="Music")
response = client.scan(TableName="Music")
assert response["Items"] == scan_response["Items"]
my_dyno_pd = DynamoDBPandas(session)
dynamo_table = my_dyno_pd.get_table(table_name="Music")
assert dynamo_table.name == "Music"
my_dyno_pd.put_items(
    table_name="Music",
    item_list=[
        {"Artist": "Nobody", "SongTitle": "Not Famous"},
        {"Artist": "Nicho", "SongTitle": "Bad song"},
    ],
)
assert dynamo_table.get_item(Key={"Artist": "Nobody", "SongTitle": "Not Famous"})[
    "Item"
] == {"Artist": "Nobody", "SongTitle": "Not Famous"}
my_dyno_pd.delete_items(
    table_name="Music",
    item_list=[
        {"Artist": "Nobody", "SongTitle": "Not Famous"},
        {"Artist": "Nicho", "SongTitle": "Bad song"},
    ],
)
after_del_resp = dynamo_table.get_item(
    Key={"Artist": "Nobody", "SongTitle": "Not Famous"}
)
assert "Item" not in after_del_resp
response = client.delete_table(TableName="Music")


2022-11-30 15:04:59.389 [INFO] [credentials:1223] MainThread:botocore.credentials - Found credentials in shared credentials file: ~/.aws/credentials


### Redshift

In [ ]:
# TODO move to demo notebook
# load_dotenv()
redshift_user = os.getenv('REDSHIFT_USER')
redshift_password = os.getenv('REDSHIFT_PASSWORD')
conn = RedshiftConnector(
    iam=True,
    database='dev',
    db_user=redshift_user,
    password=redshift_password,
    user=redshift_user,
    cluster_identifier='nutrien-insights-redshift-cluster-test',
    # host=
    # 'nutrien-insights-redshift-cluster-test.cydu5eacdk8z.ca-central-1.redshift.amazonaws.com',
    # autocommit=True
)

# with conn.cursor() as cursor:
#     cursor.execute("create table category_stage (\
#         catid smallint default 0,\
#         catgroup varchar(10) default 'General', \
#         catname varchar(10) default 'General', \
#         catdesc varchar(50) default 'General');")
# print(conn.redshift_execute_select(table="category_stage"))

# print(conn.redshift_execute_select(table='event'))

# conn.redshift_execute_insert(
#     table='category_stage',
#     source_table='category',
#     source_columns='catid, catname',
#     values="(12, 'Concerts', 'Comedy', 'All stand-up comedy performances')"
# )

# print(
#     conn.redshift_execute_delete(
#         table="category",
#         reference_tables="event",
#         conditions="event.catid=category.catid and category.catid=9"))

# print(conn.redshift_execute_select(table='category_stage'))
#Connect to the cluster
# conn = redshift_connector.connect(
#     host=
#     'nutrien-insights-redshift-cluster-test.cydu5eacdk8z.ca-central-1.redshift.amazonaws.com',
#         database='dev',
#         user='awsuser',
#         password='Nutrieninsights1'
#   )


### Config

In [ ]:

json_text = """
{
"fruit": {
    "apple": {
        "size": "small",
        "color": "red",
        "country": "USA"
    },
    "banana":{
        "size": "medium",
        "color": "yellow",
        "country": "Fiji"
    },
    "orange":{
        "size": "large",
        "color": "orange",
        "country": "Egypt"
    }
}
}
"""
yaml_text = """
    name: "Vishvajit"
    age: 23
    address: Noida
    Occupation: Software Developer
    Skills:
    - Python
    - Django
    - Flask
    - FastAPI
    - DRF ( Django Rest Framework )
"""
toml_text = """
    [user]
    player_x.color = "blue"
    player_o.color = "green"

    [constant]
    board_size = 3

    [server]
    url = "https://tictactoe.example.com"
"""
ini_text = """
    [apple]
    size = small
    color = red
    country = USA
"""
sample_dict = {
    "Vegetable": {
        "carrot": {"size": "cylindrical", "color": "orange", "country": "America"}
    }
}
a = ConfigParser(json_text)
b = ConfigParser(ini_text)
c = ConfigParser(yaml_text)
d = ConfigParser(toml_text)
print(a.merge_dictionaries(sample_dict))
# print(c.parse_yaml())
# print(d.parse_toml())
# print(b.parse_ini())

# from pynutrien.aws.s3 import S3Operations
# s3 = S3Operations()
# s3.read_object()

import s3fs
s3 = s3fs.S3FileSystem(anon=False)
with s3.open(path) as fh:
    data = json.load(fh)


### Logger

In [32]:

from time import sleep

logger = Logger(__name__)
logger2 = Logger()

assert logger is logger2

logger.info("Hello")
logger.warning("Hello")

with RuntimeLogger(logger, "time_this"):
    sleep(0.1)

# This should not be done, since it will print the traceback/error for an exception that is handled
try:
    with RuntimeLogger(logger, "time_this_2"):
        sleep(0.1)
        raise RuntimeError("oops")
        sleep(0.1)
except RuntimeError:
    pass

# when debugging, this will print the traceback once for stderr and once for stdout
# in the glue environment it will be split into the different output streams
with RuntimeLogger(logger, "time_this_3"):
    sleep(0.1)
    raise RuntimeError("oops2")
    sleep(0.1)


NameError: name 'Logger' is not defined

### ETL

In [ ]:


class TestETL1(ETLExtendedBase):
    job_name = "TEST_ETL_1"

    def setup(self):
        pass

    def extract(self):
        pass

    def transform(self):
        pass

    def load(self):
        pass

class TestETL2(ETLExtendedBase):
    job_name = "TEST_ETL_2"

    # def job_name(self): pass

    def extract(self):
        pass

    def transform(self):
        raise Exception("Test")

    def load(self):
        pass

j1 = TestETL1()
j1.run()

j2 = TestETL2()
j2.run()


### Column

In [34]:

# There're 4 scenarios for column names to appear in a dataset
# 1. Dataset has headers (read from csv as "string" even when it's integer value) -> should work normally
# 2. Dataset has headers (directly created from pandas so when it's integer, column name is read as "integer")
#     -> a TypeError will be raised by the system
# 3. Dataset has no headers (read from csv as "string" ("Unnamed")) -> column names become "Unnamed" and work normally
# 4. Dataset has no headers (directly created from pandas so an integer will be assigned automatically to name)
#     -> a TypeError will be raised by the system

# import pandas as pd
# df = pd.read_csv("tests/std_column_names_test_files/bigmac.csv")
# df = pd.DataFrame([[0,423,2342],[12,123,1231],[1,3,5]])
# df = pd.DataFrame({
#     0: ['a','b'],
#     1: ['c','d'],
#     'price': [2,4]
# # })
# print(df)
# df.columns = df.columns.astype(str)
# print(type(df.columns))
# print(df.columns.dtype)
# print(list(df))
# print(standardize_column_names(df))
import pyspark.sql.types as T
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark_df_with_integer_headers = spark.createDataFrame(data=[
        ('a','c',2),
        ('b','d',4)
    ],
    schema = T.StructType([
    T.StructField('1         ', T.StringType(), False),
    T.StructField('2', T.StringType(), False),
    T.StructField('Price   ', T.IntegerType(), False),
]))

new_df = standardize_column_names(spark_df_with_integer_headers)
print(type(new_df))
new_df.printSchema()


NameError: name 'standardize_column_names' is not defined

In [ ]:
# TODO move
# if __name__ == '__main__':
# date_range = pd.date_range('2022-11-04', periods=5, freq='D')
# test_files = {
#         'id': [1, 2, 3, 4, 5],
#         'letter_code': ['A','B','C','D','E'],
#         'string_date': ['2022-11-04','2022-11-05','2022-11-06','2022-11-07','NaN'],
#         'date': date_range
#         }
# df = pd.DataFrame(test_files)
# print(df)

# df['date'] = df['date'].dt.strftime('%B %d, %Y')
# new_df = delegate_df(df=df, date_column='date', new_format='%B %d, %dxfcsz')
# print(new_df)
# from pyspark.sql import SparkSession
# import pyspark.sql.types as T
# from datetime import date
# spark = SparkSession.builder.appName('pytest-spark').getOrCreate()
# test_files = [
#     (1,'A','2022-11-04',date(2022,11,4)),
#     (2,'B','2022-11-05',date(2022,11,5)),
#     (3,'C','2022-11-06',date(2022,11,6)),
#     (4,'D','2022-11-07',date(2022,11,7)),
#     (5,'E','2022-11-08',date(2022,11,8)),
#     ]
# schema = T.StructType([
#     T.StructField('id', T.StringType(), False),
#     T.StructField('letter_code', T.StringType(), False),
#     T.StructField('string_date', T.StringType(), False),
#     T.StructField('date', T.DateType(), False)
# ])
# df = spark.createDataFrame(test_files=test_files, schema=schema)
# df.show()
# date_df = df.select("date").collect()
# date_values = [e.__getattr__("date").strftime('%B %d, %Y') for e in date_df]
# print(date_values)
# df = spark.createDataFrame([('2015-04-08',)], ['dt'])
# new = df.withColumn('date',date_format('dt', 'MM/dd/yyy'))
# new.show()
# new_df = delegate_df(df=df, date_column='string_date',new_format='%B %d, %dxfcsz')
# str_date_col = new_df.select("string_date").collect()
# delated_date_str_list = [e.__getattr__('string_date') for e in str_date_col]
# new_df.show()
# print(delated_date_str_list)
# des = [
#     'November 04, 2022',
#     'November 05, 2022',
#     'November 06, 2022',
#     'November 07, 2022',
#     'November 08, 2022']
# print(delated_date_str_list == des)

# obj_w_date = 'abc'
# print(type(date(2022,11,4)))
# delegated_obj_w_date = delegate_df(df=obj_w_date, new_format='%B %d, %Y')
# print(delegated_obj_w_date)


In [ ]:

# TODO remove
import json
res = json.load('s3://aws-glue-assets-175633476877-ca-central-1/config/env.json')